<a href="https://colab.research.google.com/github/tetrar124/Safety_Science/blob/master/TUPtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%pip install tensorflow==1.13.1

from tensorflow.keras.callbacks import History
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/colab/
import os
#os.chdir(r'//drive/My Drive/Data/Meram Chronic Data')
import numpy as np
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/colab


In [0]:
ejectCAS = ['10124-36-4', '108-88-3', '111991-09-4', '116-29-0', '120-12-7', '126833-17-8', '13171-21-6',
                    '1333-82-0', '137-30-4', '148-79-8', '1582-09-8', '1610-18-0', '2058-46-0', '2104-64-5',
                    '21725-46-2',
                    '2303-17-5', '25311-71-1', '25812-30-0', '298-00-0', '298-04-4', '314-40-9', '330-54-1',
                    '4170-30-3',
                    '4717-38-8', '50-00-0', '52645-53-1', '55406-53-6', '56-35-9', '56-38-2', '60207-90-1', '6051-87-2',
                    '62-53-3', '6317-18-6', '69-72-7', '7440-02-0', '7447-40-7', '7722-84-1', '7733-02-0', '7758-94-3',
                    '80844-07-1', '82657-04-3', '84852-15-3', '86-73-7', '9016-45-9', '99-35-4']

df =pd.read_csv('fishMorganMACCS.csv')
#df2=pd.read_csv('chronicMACCSKeys_tanimoto.csv')
#df2 = df2.drop(ejectCAS,axis=1).set_index('CAS').dropna(how='all', axis=1)
baseDf = df
extractDf =  df['CAS'].isin(ejectCAS)
df = df[~df['CAS'].isin(ejectCAS)]
#df = df.set_index('CAS')
#df = pd.concat([df,df2],axis=1, join_axes=[df.index]).reset_index()
y = df['logTox']
#dropList = ['CAS','toxValue','logTox','HDonor', 'HAcceptors', 'AromaticHeterocycles', 'AromaticCarbocycles', 'FractionCSP3']
dropList = ['CAS','toxValue','logTox']
X = df.drop(columns=dropList)
#Normalize
def normalize(X):
    changeList = []
    for i,name in enumerate(X.columns):
        if i <679:
            changeList.append((0,1))
        elif i > 692:
            changeList.append((0,1))
        else:
            #try:
            #name = float(name)
            #except:
            std =X[name].std()
            mean = X[name].mean()
            if std==0:
              X[name] = X[name]
            else:
              X[name] = X[name].apply(lambda x: ((x - mean) * 1 / std + 0))
            changeList.append((mean, std))
    return X, changeList
X2,_ = normalize(X)
X2 = np.array(X)
y2 = np.array(y)

In [11]:
from tensorflow.contrib.tpu.python.tpu import keras_support
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import time
import pandas as pd

import numpy as np
import itertools
import pandas as pd

from lightgbm import LGBMRegressor
#from fastFM import sgd,als

from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.cross_decomposition import PLSRegression

from sklearn.model_selection import KFold, train_test_split
from sklearn.model_selection import GridSearchCV,  cross_validate, StratifiedKFold
from sklearn.pipeline import make_pipeline

from mlxtend.regressor import StackingRegressor
from mlxtend.feature_selection import ColumnSelector
#from rgf.sklearn import RGFRegressor
import xgboost
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.datasets import load_boston

import tensorflow as tf
import os
from PIL import Image
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing.image import array_to_img
from tensorflow.python.keras.preprocessing.image import img_to_array
from tensorflow.python.keras.preprocessing.image import load_img
from tensorflow.python.keras.layers import Conv1D, MaxPooling1D,BatchNormalization,concatenate,add,MaxPooling1D,BatchNormalization
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Input, Dense, LSTM,CuDNNLSTM,LSTM,Concatenate,Add,Embedding
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.python.keras import applications
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import accuracy_score
from tensorflow.python.keras import regularizers

from sklearn.base import BaseEstimator, TransformerMixin,RegressorMixin
from sklearn.pipeline import make_pipeline

import scipy as sp
from sklearn.preprocessing import normalize
from sklearn.datasets import load_boston
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


In [4]:
(X_train, y_train), (_, _) = mnist.load_data()
X_train = X_train / 255.0
y_train = to_categorical(y_train)

input = Input((28,28))
x = Flatten()(input)
x = Dense(64, activation="relu")(x)
x = Dense(10, activation="softmax")(x)
model = Model(input, x)
model.compile(Adam(), loss="categorical_crossentropy", metrics=["acc"])

# history = model.fit(X_train, y_train, epochs=1, batch_size=1024)
# print(history)
# print(history.history)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
def CNN_model(data= X2):
  inputs = Input(((data.shape[1],1)))
#   print(inputs)
  x = Conv1D(8,16,padding="same", activation="relu")(inputs)
  x = Conv1D(16,32,padding="same", activation="relu")(x)
  x = Conv1D(32,64,padding="same", activation="relu")(x)
  x = MaxPooling1D(pool_size=2, strides=None, padding='same', data_format='channels_last')(x)
  #x = BatchNormalization()(x)

  x = Flatten()(x)
  x = Dense(256, activation="relu",kernel_regularizer=regularizers.l2(0.002))(x)
  x = Dropout(0.5)(x)
  x = Dense(128, activation="relu",kernel_regularizer=regularizers.l1(0.002))(x)
  x = Dropout(0.5)(x)
  x = Dense(128, activation="relu",kernel_regularizer=regularizers.l1(0.001))(x)
  #x = BatchNormalization()(x)

  predictions = Dense(1)(x)
  model = Model(inputs=inputs, outputs=predictions)
  model.compile(optimizer="adam",loss='mean_squared_error')
  #model.summary()
  
  return model

In [0]:
model =  CNN_model()
tpu_grpc_url = "grpc://"+os.environ["COLAB_TPU_ADDR"]
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu_grpc_url)
strategy = keras_support.TPUDistributionStrategy(tpu_cluster_resolver)
model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=strategy)
from tensorflow.keras.callbacks import History

cv = KFold(n_splits=10, shuffle=True, random_state=0)

def calcCorr(real, pred):
    corr = np.corrcoef(real, pred.flatten())[0, 1]
    return corr
def calcRMSE(real, pred):
  RMSE = (np.sum((pred - real.tolist()) ** 2) / len(pred)) ** (1 / 2)
  return RMSE
for i,(train, test) in enumerate(cv.split(X2)):
  trainX = np.expand_dims(X2[train], axis=2)
  testX = np.expand_dims(X2[test], axis=2)
  print(trainX.shape,y2[train].shape)
  history = History()
  es = EarlyStopping(monitor='val_loss', patience=50, verbose=1, mode='auto')

  model.fit(trainX,y2[train], epochs=100, batch_size=1024, callbacks=[history,es],validation_data=(testX, y2[test]),metric=['mean_squared_error'])
  print(history)
  print(history.history)

INFO:tensorflow:Querying Tensorflow master (grpc://10.69.194.218:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 6094625470408295852)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 13812256486755107459)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 8406601832579415794)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 1004065896015030234)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 6126146737264658958)
INFO:tensorflow:*** Available Device: _DeviceAttributes(

In [28]:
print(history.history)

{'loss': [0.7997977596347157, 0.7719685196137407, 0.7581331066793842, 0.7740553484436049, 0.7477255228665997, 0.7354119023567179, 0.7262272563943618, 0.733360590556747, 0.7637749835386859, 0.7441250194586728, 0.74465433151137, 0.7466241491907355, 0.7201752437755425, 0.7111752377979728, 0.7191629214451523, 0.7142495187222165, 0.7065404571190252, 0.6963046230606741, 0.6930500642191108, 0.7042077711665388, 0.7133742831152451, 0.7131790258485305, 0.7128970899657933, 0.6710902465139479, 0.6792159354507237, 0.6901233624947398, 0.7131350724648542, 0.6970869624794216, 0.6973501224239069, 0.7141127342455571, 0.6980438826356555, 0.6933444253737363, 0.7032278167451769, 0.6875871926202934, 0.7287406360184439, 0.6954431837908033, 0.6781644969518676, 0.6743428091542504, 0.6787605581819959, 0.6828178356346354, 0.7067594055690208, 0.68098198332757, 0.7030758573166346, 0.7119648858865243, 0.6878438072990167, 0.6943793200728532, 0.7060373204803974, 0.686694804074175, 0.6978471027843924, 0.69859900941249